# Finding bigfoot (partie 2)

In [ ]:
team_name = "qwertyuio"

In [ ]:
import Pkg; Pkg.activate(pwd())
_code_path = joinpath(dirname(Base.active_project()), "code")
modules = ["pkg", "palettes", "confusionmatrix", "mocks", "crossvalidation", "pipelines", "nbc", "variableselection", "bootstrap"]
for m in modules
    include(joinpath(_code_path, "$(m).jl"))
end

In [ ]:
JLD2.jldopen("training.jld2", "r") do file
    global X = file["X"]
    global y = file["y"]
end;
presences = findall(y);
absences = findall(.!y);

## Définition du modèle

In [ ]:
model = SDM(RawData(), NBC(), Thresholder())

Bagging

In [ ]:
bags = bootstrap(y, X; n=10)
ensemble = Bagging(model, bags)
train!(ensemble, y, X; classify=true)

First map with the ensemble using all variables

In [ ]:
predictors = [SpeciesDistributionToolkit._read_geotiff("layers.tiff", SimpleSDMPredictor; bandnumber=v) for v in 1:19]
prediction = similar(predictors[1])
uncertainty = similar(predictors[1])
Threads.@threads for k in keys(prediction)
    input = [predictor[k] for predictor in predictors]
    prediction[k] = predict(ensemble, input; classify=false, consensus=median)
    uncertainty[k] = predict(ensemble, input; classify=false, consensus=std)
end

Visualisation du modèle

Validation croisée

In [ ]:
splits = kfold(y, X; k=10)